In [1]:
#文本分类实例

In [1]:
from tokenize import tokenize

from seaborn import load_dataset
#导入相关包
from transformers import AutoTokenizer, AutoModelForSequenceClassification,TrainingArguments, Trainer

In [6]:
#加载数据
import pandas as pd
data = pd.read_csv('./data.csv')
data.head()

,label,review
0,1,很快，好吃，味道足，量大
1,1,没有送水没有送水没有送水
2,1,非常快，态度好。
3,1,方便，快捷，味道可口，快递给力
4,1,菜味道很棒！送餐很及时！


In [7]:
#数据处理
data = data.dropna()#处理空数据
data

,label,review
0,1,很快，好吃，味道足，量大
1,1,没有送水没有送水没有送水
2,1,非常快，态度好。
3,1,方便，快捷，味道可口，快递给力
4,1,菜味道很棒！送餐很及时！
...,...,...
11982,0,以前几乎天天吃，现在调料什么都不放，
11983,0,昨天订凉皮两份，什么调料都没有放，就放了点麻油，特别难吃，丢了一份，再也不想吃了
11984,0,"凉皮太辣,吃不下都"
11985,0,本来迟到了还自己点！！！


In [33]:
#创建dataset
import torch
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv('./data.csv')
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"],self.data.iloc[index]["label"]#.iloc[index]选择第index行，[review] 选中行中获取review列的值
    def __len__(self):
        return len(self.data)
tokenizer = AutoTokenizer.from_pretrained('D:/Hugging Face Hub/test2')
def collate_fn(batch):
    texts,labels =[],[]
    for i in batch:
        texts.append(i[0])
        labels.append(i[1])
    inputs = tokenizer(texts,max_length=128,padding="max_length",truncation=True,return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs


In [34]:
dataset = MyDataset()


In [35]:
#划分数据集和验证集
from torch.utils.data import random_split
trainset,validset = random_split(dataset,lengths=[0.9,0.1])


In [36]:
#DateLoader
from torch.utils.data import DataLoader
trainloader = DataLoader(dataset=trainset,
                         batch_size=32,
                         shuffle=True,collate_fn=collate_fn)
validloader = DataLoader(dataset=validset,batch_size=64,shuffle=False,collate_fn=collate_fn)


In [37]:
#创建模型和优化器
from torch.optim import Adam
model = AutoModelForSequenceClassification.from_pretrained('D:/Hugging Face Hub/test2')
optimizer = Adam(model.parameters(), lr=2e-5)
if torch.cuda.is_available():
    model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at D:/Hugging Face Hub/test2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
#训练和验证
def train(epoch=5,log_step=100):
    global_step=0
    for epoch in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items() }
            output = model(**batch)
            optimizer.zero_grad()
            output.loss.backward()
            optimizer.step()
            global_step += 1
            if global_step % log_step == 0:
                print(f'epoch:{epoch} step:{global_step} loss:{output.loss.item()}')
        acc = evaluate()
        print(f'epoch:{epoch} acc:{acc}')
def evaluate():
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items() }
            output = model(**batch)
            pred =torch.argmax(output.logits, dim=-1)
            correct += pred.eq(batch["labels"]).sum().item()
    return correct / len(validset)


In [46]:
#模型训练
train()


epoch:0 step:100 loss:0.3779287040233612
epoch:0 step:200 loss:0.27839696407318115
epoch:0 step:300 loss:0.23721197247505188
epoch:0 acc:0.9065108514190318
epoch:1 step:400 loss:0.23831325769424438
epoch:1 step:500 loss:0.16930896043777466
epoch:1 step:600 loss:0.3442832827568054
epoch:1 acc:0.9131886477462438
epoch:2 step:700 loss:0.07657378911972046
epoch:2 step:800 loss:0.3693259358406067
epoch:2 step:900 loss:0.18259668350219727
epoch:2 step:1000 loss:0.21222908794879913
epoch:2 acc:0.9081803005008348
epoch:3 step:1100 loss:0.02829211950302124
epoch:3 step:1200 loss:0.1568707972764969
epoch:3 step:1300 loss:0.21479718387126923
epoch:3 acc:0.9065108514190318
epoch:4 step:1400 loss:0.04881398007273674
epoch:4 step:1500 loss:0.05350301042199135
epoch:4 step:1600 loss:0.23297372460365295
epoch:4 acc:0.9106844741235393


In [48]:
#模型预测
sen = "这家店的菜吃着还行，下次还会点"
model.eval()
with torch.no_grad():
    inputs = tokenizer(sen,return_tensors="pt")
    inputs = {k:v.cuda() for k,v in inputs.items() }
    output = model(**inputs)
    pred = torch.argmax(output.logits, dim=-1)
    if pred.item()==1:
        print("good")
    else:
        print("bad")



good


In [49]:
from transformers import pipeline
pipe = pipeline("text-classification",model='D:/Hugging Face Hub/test2')
#model.config.id2label = id2_label

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at D:/Hugging Face Hub/test2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [50]:
pipe(sen)

[{'label': 'LABEL_1', 'score': 0.6045428514480591}]

In [52]:
    #加载数据集
from datasets import load_dataset
dataset = load_dataset("csv",data_files="./data.csv",split="train")
dataset = dataset.filter(lambda x:x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 11987
})

In [ ]:
#划分数据集
dataset_train = dataset.train_test_split(test_size=0.1)
#创建dataloader
def process_data(examples):
    tokenizer_dataset = tokenizer(examples["review"],max_length=128,padding="max_length",truncation=True,return_tensors="pt")
    tokenizer_dataset["label"] = examples["label"]
    return tokenizer_dataset
tokenizer_dataset = dataset.map(process_data,batcher=True,remove_columns = dataset["train"].column_names)


In [7]:
#创建评估函数
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def eval_metric(eval_predict):
    prediction,labels = eval_predict
    prediction = prediction.argmax(dim=-1)
    acc = acc_metric.compute(prediction=prediction,labels=labels)
    f1 = f1_metric.compute(prediction=prediction,labels=labels)
    acc.update(f1)
    return acc



In [8]:
#创TrainingArguments(定义训练的规则和配置)
train_args = TrainingArguments(output_dir="./checkpoints")

In [ ]:
#创建Trainer（执行训练的引擎和管家）
trainer = Trainer(model= model,
                  args=train_args,
                  train_dataset=tokenizer_dataset["train"],
                  eval_dataset=tokenizer_dataset["test"],
                  data_collator=collate_fn,
                  compute_metrics=eval_metric)

In [ ]:
#模型训练
trainer.train()
#模型评估
trainer.evaluate()#里面可以放数据集